# IDAES Modeling

Docs: https://idaes-pse.readthedocs.io/en/stable/index.html
Video: https://www.youtube.com/channel/UCpp3J_990C0Oz_CbxRDfr6g


## IDAES Flash Tutorial

In [2]:
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value
from idaes.core import FlowsheetBlock
import idaes.logger as idaeslogger
from idaes.generic_models.properties.activity_coeff_models.BTX_activity_coeff_VLE import BTXParameterBlock
from idaes.generic_models.unit_models import Flash
from idaes.core.util.model_statistics import degrees_of_freedom

# create model instance
m = ConcreteModel()

# add a flowsheet block
m.fs = FlowsheetBlock(default={"dynamic": False}) 

# specify physical properties package
m.fs.properties = BTXParameterBlock(default={"valid_phase": ('Liq', 'Vap'),
                                            "activity_coeff_model": "Ideal",
                                            "state_vars": "FTPz"})

# specify flash unit
m.fs.flash = Flash(default={"property_package": m.fs.properties})

# inlet conditions
m.fs.flash.inlet.flow_mol.fix(1.0)
m.fs.flash.inlet.temperature.fix(368.0)
m.fs.flash.inlet.pressure.fix(101325.0)
m.fs.flash.inlet.mole_frac_comp[0, "benzene"].fix(0.5)
m.fs.flash.inlet.mole_frac_comp[0, "toluene"].fix(0.5)

# flash unit specifications
m.fs.flash.heat_duty.fix(0.0)
m.fs.flash.deltaP.fix(0.0)

# verify all degrees of freedom have been fully specified
print("Degrees of Freedom =", degrees_of_freedom(m))

# initialize flash unit
m.fs.flash.initialize(outlvl=idaeslogger.INFO)

# solve
solver = SolverFactory("ipopt")
status = solver.solve(m, tee=False)

# prepare report
m.fs.flash.report()

Degrees of Freedom = 0
2022-02-21 19:22:52 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 1 optimal - Optimal Solution Found.
2022-02-21 19:22:53 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 2 optimal - Optimal Solution Found.
2022-02-21 19:22:53 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 3 optimal - Optimal Solution Found.
2022-02-21 19:22:53 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 4 optimal - Optimal Solution Found.
2022-02-21 19:22:54 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 5 optimal - Optimal Solution Found.
2022-02-21 19:22:54 [INFO] idaes.init.fs.flash.control_volume.properties_out: Initialization Step 1 optimal - Optimal Solution Found.
2022-02-21 19:22:54 [INFO] idaes.init.fs.flash.control_volume.properties_out: Initialization Step 2 optimal - Optimal Solution Found.
2022-02-21 19:22:55 [INFO] idaes.init.fs.fla

In [34]:
m.fs.visualize("")

Loading saved flowsheet from '.json'
Saving flowsheet to default file '.json' in current directory (/Users/jeff/Google Drive/GitHub/MO-book/work-in-progress)
Flowsheet visualization at: http://localhost:61434/app?id=


In [35]:
print('Vap. Outlet Temperature = ', value(m.fs.flash.vap_outlet.temperature[0]))

Vap. Outlet Temperature =  368.0


In [36]:
m.fs.flash.vap_outlet.temperature.display()
m.fs.flash.vap_outlet.display()

_Vap_temperature_ref : Size=1, Index=fs._time, ReferenceTo=fs.flash.control_volume.properties_out[:].component('temperature')
    Key : Lower : Value : Upper : Fixed : Stale : Domain
    0.0 :     0 : 368.0 :  None : False : False : NonNegativeReals
vap_outlet : Size=1
    Key  : Name           : Value
    None :       flow_mol : {0.0: 0.3961181748774182}
         : mole_frac_comp : {(0.0, 'benzene'): 0.6339766485081291, (0.0, 'toluene'): 0.3660233514918707}
         :       pressure : {0.0: 101325.0}
         :    temperature : {0.0: 368.0}


In [53]:
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value
from idaes.core import FlowsheetBlock
import idaes.logger as idaeslogger
from idaes.generic_models.properties.activity_coeff_models.BTX_activity_coeff_VLE import BTXParameterBlock
from idaes.generic_models.unit_models import Product
from idaes.core.util.model_statistics import degrees_of_freedom

m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False}) 

m.fs.properties = BTXParameterBlock(default={"valid_phase": ('Liq', 'Vap'),
                                            "activity_coeff_model": "Ideal",
                                            "state_vars": "FTPz"})

m.fs.product = Product(default={"property_package": m.fs.properties})

m.fs.product.inlet.temperature.fix(368.0)
m.fs.product.inlet.pressure.fix(101325.0)


print("Degrees of Freedom =", degrees_of_freedom(m))


Degrees of Freedom = 3
